In [148]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [149]:
page = requests.get("https://www.aldi-sued.de/de/sortiment/bio/")

In [142]:
soup = BeautifulSoup(page.content,'html.parser')

In [144]:
def get_product_name(productblocks,i):
    name = productblocks[i].find("div",class_="box--description--header").get_text().strip()
    return name

In [197]:
def get_product_price(productblocks,i):
    value= productblocks[i].find('span',class_="box--value").get_text()
    value = re.sub("[^0-9]", "",value)
    if value=="":
        value = '0'
    if productblocks[i].find('span',class_="box--decimal"):
        decimal = productblocks[i].find('span',class_="box--decimal").get_text()
    else:
        decimal = productblocks[i].find_all('span',class_="box--value")[1].get_text()
    decimal = re.sub("[^0-9]", "",decimal)
    if decimal == '':
        decimal = '0'
    if value is None:
        value = '0'
    if decimal is None:
        decimal = '0'
    price = float(value)+float(decimal)*0.01
    return price

In [198]:
def sub_category_title(category,i):
    sub = category.find("h2",class_="subheader-blue")
    sub_title = sub.get_text().strip()
    return sub_title

In [199]:
subcat = soup.find_all("div",{"id":re.compile("^c......*"),"class":"csc-default"})
len(subcat)

22

In [200]:
dataset = pd.DataFrame(columns=['category_title','name','price'])

In [201]:
for i in range(2,len(subcat)):
    subdict={}
    if subcat[i].find("h2",class_="subheader-blue"):
        category_title = subcat[i].find("h2",class_="subheader-blue").get_text()
    else: break
    product_group = subcat[i].find("div",class_="tx-aldi-products")
    products = product_group.find_all("div",class_="ratio-container m-ratio-57x25 box--description m-no-ratio-on-phone")
    for j in range(len(products)):
        subdict['category_title'] = category_title
        subdict['name'] = get_product_name(products,j)
        subdict['price']= get_product_price(products,j)
        dataset.loc[j,:] = subdict
        #print(subdict)
#print(maindict)

In [202]:
dataset


,category_title,name,price
0,Alkoholische Getränke,Bio-Cuvée rot Rheinhessen / Pfalz QbA,2.99
1,Alkoholische Getränke,Bio-Cuvée weiss Rheinhessen / Pfalz QbA,2.99
2,Alkoholische Getränke,Bio-Airén y Macabeo La Mancha DO,2.69
3,Nicht-alkoholische Getränke,AMAROY® Premium Röstkaffee Bio Kaffee,3.75
4,Nicht-alkoholische Getränke,Amaroy Fairer Biokaffee,4.79
5,Nicht-alkoholische Getränke,AMAROY®Fairer Biokaffee Kaffee Pads,1.79
6,Nicht-alkoholische Getränke,ONE WORLD® Bio-Trinkschokolade,2.49
7,Nicht-alkoholische Getränke,ONE WORLD® Fairtrade Bio Tee,1.59
8,Nicht-alkoholische Getränke,bio Tee,1.09
9,Molkereiprodukte,bio Joghurt,0.85
